## Create a mount point

In [ ]:
###### Mount Point 1 through Oauth security.
storageAccount = "gen10datafund2205"
storageContainer = "capstone-group6-producer-data"
clientSecret =
clientid = 
mount_point = "/mnt/gavanvanover/capstone-group6-producing"
#20200906-20201006/Detroit911-20200906-20201006.csv
#@28077959-1e7b-474a-995f-492292e0b7f8
#Copy to clipboard

configs = {"fs.azure.account.auth.type": "OAuth",
       "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
       "fs.azure.account.oauth2.client.id": clientid,
       "fs.azure.account.oauth2.client.secret": clientSecret,
       "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d46b54b2-a652-420b-aa5a-2ef7f8fc706e/oauth2/token",
       "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

try: 
    dbutils.fs.unmount(mount_point)
except:
    pass

dbutils.fs.mount(
source = "abfss://"+storageContainer+"@"+storageAccount+".dfs.core.windows.net/",
mount_point = mount_point,
extra_configs = configs)

/mnt/gavanvanover/capstone-group6-producing has been unmounted.
Out[1]: True

In [ ]:
display(dbutils.fs.ls("/mnt/gavanvanover/capstone-group6-producing"))

path,name,size,modificationTime
dbfs:/mnt/gavanvanover/capstone-group6-producing/retail_census_data_NAICS_NAPCS_2017.csv,retail_census_data_NAICS_NAPCS_2017.csv,675608,1660105900000


## Census Dataset

In [ ]:
df = spark.read.csv('/mnt/gavanvanover/capstone-group6-producing/retail_census_data_NAICS_NAPCS_2017.csv', header = 'True')
display(df.dtypes)

_1,_2
Geographic Area Name (NAME),string
Year (YEAR),string
2017 NAPCS collection code (NAPCS2017),string
Meaning of NAPCS collection code (NAPCS2017_LABEL),string
2017 NAICS code (NAICS2017),string
Meaning of NAICS code (NAICS2017_LABEL),string
Meaning of Type of operation code (TYPOP_LABEL),string
Number of establishments (ESTAB),string
"Sales, value of shipments, or revenue of NAPCS collection code ($1,000) (NAPCSDOL)",string
"Industry contribution to total NAPCS collection code sales, value of shipments, or revenue (%) (NAICSALL_PCT)",string


In [ ]:
import pyspark.sql.functions as f
from pyspark.sql.types import IntegerType

In [ ]:
produce_df = df['2017 NAPCS collection code (NAPCS2017)', 'Meaning of NAPCS collection code (NAPCS2017_LABEL)', '2017 NAICS code (NAICS2017)', 'Meaning of NAICS code (NAICS2017_LABEL)', 'Number of establishments (ESTAB)','Sales, value of shipments, or revenue of NAPCS collection code ($1,000) (NAPCSDOL)']

# For the sales column replace D and A with Null
producer_df = produce_df.withColumn("Sales, value of shipments, or revenue of NAPCS collection code ($1,000) (NAPCSDOL)", \
       f.when((f.col('Sales, value of shipments, or revenue of NAPCS collection code ($1,000) (NAPCSDOL)')=="D") | (f.col('Sales, value of shipments, or revenue of NAPCS collection code ($1,000) (NAPCSDOL)')=="A") , None) \
          .otherwise(f.col("Sales, value of shipments, or revenue of NAPCS collection code ($1,000) (NAPCSDOL)")))

# For the number of establishments column replace D and S with Null
producer_df2 = producer_df.withColumn("Number of establishments (ESTAB)", \
       f.when((f.col('Number of establishments (ESTAB)')=="D") | (f.col('Number of establishments (ESTAB)')=="S") , None) \
          .otherwise(f.col("Number of establishments (ESTAB)")))

# Remove the (s) from the end of numbers in the sales column, remove the commas from both number columns
producer_df3 = producer_df2.withColumn("NEWSales", f.regexp_replace(f.col("Sales, value of shipments, or revenue of NAPCS collection code ($1,000) (NAPCSDOL)"), "[, (s)]", ""));
producer_df3 = producer_df3.withColumn("NEWEstab", f.regexp_replace(f.col("Number of establishments (ESTAB)"), "[,]", ""));

# Drop the old columns
producer_df4 = producer_df3['2017 NAPCS collection code (NAPCS2017)', 'Meaning of NAPCS collection code (NAPCS2017_LABEL)', '2017 NAICS code (NAICS2017)', 'Meaning of NAICS code (NAICS2017_LABEL)', 'NEWEstab','NEWSales']

# Change the Sales adn Establishment columns to ints and rename the new columns to the correct names
producer_df5 = (producer_df4
    .withColumn('NEWSales', producer_df4.NEWSales.cast(IntegerType()))
    .withColumn('NEWEstab', producer_df4.NEWEstab.cast(IntegerType()))
    .withColumnRenamed('NEWSales', 'Sales, value of shipments, or revenue of NAPCS collection code ($1,000) (NAPCSDOL)')
    .withColumnRenamed('NEWEstab', 'Number of establishments (ESTAB)'))

producer_df5.dtypes

Out[5]: [('2017 NAPCS collection code (NAPCS2017)', 'string'),
 ('Meaning of NAPCS collection code (NAPCS2017_LABEL)', 'string'),
 ('2017 NAICS code (NAICS2017)', 'string'),
 ('Meaning of NAICS code (NAICS2017_LABEL)', 'string'),
 ('Number of establishments (ESTAB)', 'int'),
 ('Sales, value of shipments, or revenue of NAPCS collection code ($1,000) (NAPCSDOL)',
 'int')]

## Create the producer

In [ ]:
# Define functions that handles error messages
def error_cb(err):
    """ The error callback is used for generic client errors. These
        errors are generally to be considered informational as the client will
        automatically try to recover from all errors, and no extra action
        is typically required by the application.
        For this example however, we terminate the application if the client
        is unable to connect to any broker (_ALL_BROKERS_DOWN) and on
        authentication errors (_AUTHENTICATION). """

    print("Client error: {}".format(err))
    if err.code() == KafkaError._ALL_BROKERS_DOWN or \
       err.code() == KafkaError._AUTHENTICATION:
        # Any exception raised from this callback will be re-raised from the
        # triggering flush() or poll() call.
        raise KafkaException(err)


def acked(err, msg):
    """ 
        Error callback is used for generic issues for producer errors. 
        
        Parameters:
            err (err): Error flag.
            msg (str): Error message that was part of the callback.
    """
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg), str(err)))
    else:
        print("Message produced: %s" % (str(msg)))

In [ ]:
from confluent_kafka import Consumer
from time import sleep
import uuid
from confluent_kafka import Producer, Consumer, KafkaError, KafkaException
import json

#KAFKA variables, Move to the OS variables or configuration
# This will work in local Jupiter Notebook, but in a databrick, hiding config.py is tougher. 
confluentClusterName = "stage3talent"
confluentBootstrapServers = 
confluentTopicName = "capstone-group6"
schemaRegistryUrl = "https://psrc-gq7pv.westus2.azure.confluent.cloud"
confluentApiKey = 
confluentSecret = 
confluentRegistryApiKey = 
confluentRegistrySecret = 

#Kakfa Class Setup.
p = Producer({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,
    'group.id': str(1),  # this will create a new consumer group on each invocation.
    'auto.offset.reset': 'earliest',
    'error_cb': error_cb,
})



In [ ]:
from random import randint
from time import sleep

aDict={"name":"Gavan"}
print(confluentTopicName)

for i in range(producer_df5.count()):
    for j in producer_df5.columns:
        aDict[j] = producer_df5.collect()[i][j]
    p.produce(confluentTopicName,json.dumps(aDict))
    p.flush()
    sleep(5)

capstone-group6